# VEGAS implementation by Timur Khuzin

Sources:
1. https://e-maxx.ru/bookz/files/numerical_recipes.pdf, pp. 414-416
2. http://numerical.recipes/webnotes/nr3web9.pdf

## Implementation

In [63]:
import numpy as np
import numpy.random
from sklearn.preprocessing import normalize

def get_elems_in(matrix, column_indexes):
    r = np.ndarray(len(column_indexes))
    for row, col in enumerate(column_indexes):
        r[row] = matrix[row,col]
    return r

class Vegas:
    '''
    Model for calculation of integral using adaptive Monte-Carlo.
    '''
    def __init__(self, region, f, rand=np.random.RandomState(seed=164), steps_per_dim=1000):
        '''
        Constructor.
            region          is matrix 2xD, where D is dimension number. region[0,:] is lower bounds, region[1,:] is upper
            f               is integrated function. Must accept numpy vector with len D 
            rand            random float generator in range[0;1)
            steps_per_dim   maximal number of increments per axis
        '''
        self.f = f
        self.rand = rand
        self.steps_per_dim = steps_per_dim
        self.shift = region[0,:]
        self.region = np.copy(region)
        self.region[0, :] -= self.shift
        self.region[1, :] -= self.shift
        self.dimensions = len(region[0,:])
        
    def warm_up(self, epochs, call_per_epoch):
        '''
        Prepare field for integration.
        params:
            epochs          number of iterations of algorithm (calculations of integral)
            call_per_epoch  how many checks needed to calc integral in each iteration
        '''
        steps_per_dim = self.steps_per_dim
        
        # initiate field with equal subintervals
        field = np.zeros((self.dimensions, steps_per_dim))
        for x in range(self.dimensions):
            lattice = np.arange(0,self.region[1,x],self.region[1,x]/steps_per_dim)
            field[x, :] = lattice
        
        # weights
        probs = np.ones((self.dimensions, steps_per_dim))
        probs = normalize(probs, axis=1)
        
        total_range = self.region[1,:]
        total_volume = np.prod(total_range)
        for e in range(epochs):
            # new weights which will be used in next one
            weight_update = np.copy(probs)
            probs_reverted = 1-probs
            for c in range(call_per_epoch):
                chosen_indexes = self.rand.randint(0, self.steps_per_dim, self.dimensions)
                ranges = self.rand.rand(self.dimensions)
                subvolume = get_elems_in(probs_reverted, chosen_indexes)
                argument = get_elems_in(field, chosen_indexes) + subvolume*ranges
                argument += self.shift
                weight_change = np.abs(self.f(argument))#*np.prod(subvolume)
                for i,j in zip(range(self.dimensions), chosen_indexes):
                    weight_update[i,j] += weight_change
            probs = normalize(weight_update, axis=1)
            probs_reverted = 1 - probs
            behind_by_lattice = np.zeros(self.dimensions)
            for step in range(steps_per_dim):
                field[:, step] = behind_by_lattice
                behind_by_lattice += probs_reverted[:, step] * total_range 
        
        self.probs = probs
        self.field = field
                
        
    def vegas_integral(self, epochs, call_per_epoch):
        '''
        Implements vegas integral.
        params:
            epochs          number of iterations of algorithm (calculations of integral)
            call_per_epoch  how many checks needed to calc integral in each iteration

        returns object with fields: 
            integral        float value of calculated integral
            std_deviation   standard deviation of calculated integral
            chi2a           look Chi-squared test
        '''

        total_range = self.region[1,:]
        probs_reverted = 1-self.probs
        field = self.field
        integrals = np.zeros(epochs)
        total_volume = np.prod(total_range)
        for iteration in range(epochs):
            integr = 0.0
            for c in range(call_per_epoch):
                chosen_indexes = self.rand.randint(0, self.steps_per_dim, self.dimensions)
                ranges = self.rand.rand(self.dimensions)
                subvolume = get_elems_in(probs_reverted, chosen_indexes)
                argument = get_elems_in(field, chosen_indexes) + subvolume*ranges
                argument += self.shift
                print(argument)
                value = self.f(argument)
                if value:
                    print("Have val", value)
                integr += value
            #integr /= call_per_epoch
            print(integr)
            integrals[iteration] = integr
            print("Ended epoch %d"%iteration)
        

        # return result
        class answer: pass
        ans = answer()
        ans.integral = np.mean(integrals)
        ans.std_deviation = 0 # std_deviation
        ans.chi2a = 0 #chi2a
        return ans

In [64]:
def rnd(vect):
    if np.sum((vect-1)*(vect-1))<1:
        return 1
    return 0

model = Vegas(np.array([[0,0],[2,2]]), rnd)
print("Created")
model.warm_up(5,10000)
print("Warmed")
res = model.vegas_integral(10, 10000)
print(res.integral)

Created
Warmed
[1717.94534299 1736.08678262]
[1815.87867349  528.03915114]
[1772.23682503 1730.43352427]
[1405.71726756 1611.32455285]
[236.40693744 243.67835461]
[103.46643587 418.04341381]
[735.85047682 387.56179166]
[  99.07911441 1035.26226728]
[ 439.56570334 1625.52688796]
[  21.56570681 1524.18310982]
[199.5891801  136.01761814]
[472.52479095 470.38497698]
[  27.95122765 1001.87126172]
[1589.44388481  480.3280977 ]
[1494.62034702  364.27414039]
[ 722.45756498 1871.6799091 ]
[1488.40693344  404.71914966]
[1121.99293983  158.38845743]
[ 213.11867629 1427.88885983]
[ 436.38715179 1478.51651306]
[ 970.77253934 1090.68980924]
[1300.96238006 1286.51277317]
[ 497.43081911 1145.3682648 ]
[ 314.95625692 1227.02663062]
[1692.71954498  466.31403349]
[1303.66513189  658.86383827]
[378.63371842 932.40793058]
[1688.99189199  725.61888209]
[751.10461532 731.7158619 ]
[ 337.86475189 1153.80273561]
[1573.33736353  502.83518432]
[ 998.47804555 1186.47249322]
[917.17337758 172.01814697]
[768.479759

[1937.47004195 1752.44413328]
[668.64535931 627.95392197]
[ 19.86268903 323.72697488]
[1826.2959182  1323.29184499]
[898.32999516 245.40709926]
[814.85203208 391.27825774]
[1818.0388466   305.30095688]
[ 585.81333642 1635.33932285]
[  89.45942482 1662.57881454]
[ 992.03423211 1822.48006997]
[1528.25649742  504.61268289]
[1854.18992086 1009.5911836 ]
[1287.62638636 1008.03063855]
[1354.56652154 1084.95876207]
[ 354.9748697  1237.03992404]
[ 884.99173158 1225.67826497]
[535.87041118  49.00187646]
[1458.75070754 1688.52537845]
[1799.73409549 1221.29123671]
[1270.2035232   708.46775522]
[ 428.58817034 1301.46273242]
[1651.30139958 1213.88094126]
[605.18859748 283.97014245]
[405.84026542 897.66873199]
[1714.00699248  337.97449776]
[1869.8327819  1928.32674785]
[1462.77499693 1529.90505123]
[ 773.07550039 1589.48761237]
[689.12948189 204.79131654]
[922.61080353 491.23923122]
[1858.44483675 1382.46363205]
[1941.75345741 1504.28179114]
[1632.43701962  264.99184154]
[180.08408681 868.66075603]


[ 380.77910592 1498.59529488]
[639.41863607 800.63893582]
[1694.4936563   677.52880818]
[330.12494203 895.48021997]
[1272.08591125  795.22801516]
[689.3747978  189.53087951]
[1178.84755514 1888.05514239]
[ 659.15859137 1882.47740529]
[ 505.38647398 1315.46976315]
[420.86794094 201.33094493]
[849.82817216  42.66310188]
[470.87211015 228.15499623]
[1307.2026174   706.27269904]
[1824.42214315  170.11692657]
[ 936.07172747 1372.14739051]
[497.60212939 908.83165477]
[1692.53249113 1376.22099152]
[324.3626334  820.53886396]
[599.64741533 903.34913401]
[1786.19156502 1710.6844433 ]
[1866.27912247 1358.86112275]
[1659.13159509  225.78461754]
[346.20554278 483.95630789]
[ 858.01126045 1337.2975751 ]
[438.43917545 437.69022931]
[ 796.21706768 1615.06140313]
[ 772.35597064 1120.79432492]
[1826.47135651 1356.43458444]
[ 261.19907492 1311.10880043]
[1790.24693375 1190.21518397]
[1203.84056121  455.42995997]
[228.32021499 842.9386676 ]
[215.14461097 591.09798803]
[85.24223807 65.87457431]
[ 974.4401

[ 693.64445539 1900.28090988]
[ 767.19132477 1067.67386093]
[ 656.55758326 1850.5477016 ]
[ 62.91662712 823.74603623]
[1004.31997776 1744.46839823]
[1161.34272332  944.21071715]
[  62.71646491 1876.41773545]
[1235.04442927 1887.9100004 ]
[616.40616271 359.15042452]
[1583.77194493  903.15246078]
[1119.95460742  746.93215514]
[ 797.93573129 1182.45710067]
[1182.34953467  433.9073073 ]
[  25.81121574 1626.75569303]
[1174.39060866  959.40472791]
[1350.79053925  550.89168744]
[1556.08879413 1025.11954356]
[ 14.22369784 914.87993673]
[1197.8470181   226.54584637]
[ 290.21761014 1768.4215153 ]
[ 784.57509057 1470.76888558]
[  82.24208927 1064.09930007]
[ 416.99879864 1275.96728551]
[1507.62407432  752.86906375]
[   2.04209047 1467.13783061]
[1284.13899593  472.55247613]
[382.36793246 645.54877015]
[ 412.74004656 1486.35830931]
[1553.66787394 1447.01634687]
[ 745.35710925 1147.64440259]
[342.39205837 608.91036952]
[1056.05751825  110.96665857]
[ 399.75329241 1523.63190899]
[974.73343435 538.01

[940.03309809 903.08327755]
[  79.79684362 1244.97489669]
[1444.82629198  812.72036142]
[ 17.96689286 347.15422063]
[ 376.84194739 1376.36268347]
[1433.20657735 1321.71533828]
[888.74650303 650.97851491]
[842.52490059 298.79490484]
[1250.2423446  1051.89790453]
[1623.26445304  369.87743478]
[1026.5708565  1587.26125329]
[1657.13958037  239.50517276]
[ 819.20773112 1141.75464855]
[960.01240493  95.09074526]
[1607.59381184  449.41828979]
[426.47487862 577.85082955]
[1371.96650146 1514.69886308]
[1399.972128   1217.58991124]
[ 228.89171529 1708.71794666]
[943.65162767 118.40529298]
[139.60189216  30.94856033]
[1071.31793265 1321.767107  ]
[1877.77985782  728.07432629]
[953.75269412 555.07580056]
[493.91522302 591.33718352]
[596.03680743 959.8200697 ]
[1746.25042987  414.72859468]
[ 537.85419674 1728.18663509]
[ 990.33885292 1325.59141408]
[1519.82432478 1594.90157904]
[1246.95805964  864.39806735]
[ 890.0342629  1555.66557619]
[429.86009575 785.84375613]
[545.71195144 627.45104033]
[ 537.

...

[ 824.44162456 1225.39236023]
[1086.98248711  994.24087252]
[927.05600914 498.84049136]
[612.83327246 662.81801426]
[1725.98650562  539.88641999]
[ 116.25510584 1676.50336208]
[1441.146263    154.70042975]
[ 447.8073735  1476.58321894]
[914.62769877 930.09757506]
[650.80604363 518.67244656]
[154.98584866 293.40727587]
[280.16601627 666.55790102]
[1837.91273411 1060.30300945]
[ 586.43063626 1682.37330327]
[1467.09459727  653.41222948]
[ 440.16483694 1337.16062084]
[ 582.24297392 1393.75542611]
[1714.60568321 1756.43876627]
[ 139.68808742 1470.85494015]
[1182.63054871  256.95785585]
[1378.30685813 1939.79962976]
[728.1578388  158.82832525]
[1079.08932458  359.1267434 ]
[111.20597451 918.32467763]
[ 685.8858522  1083.22071411]
[1466.4641237  1680.99712671]
[580.31275741  21.84606274]
[ 530.33290047 1093.21498715]
[1496.56031653 1390.12211621]
[1788.25349325  516.95862757]
[1143.21370427 1419.40871051]
[ 988.78328866 1903.75359907]
[278.30248382 571.89844459]
[716.80409288 468.10169131]
[1

[734.09687851   0.8119311 ]
[1401.34452682  849.40202043]
[ 226.53692726 1681.1269044 ]
[1680.92069098 1826.60753479]
[1768.26673777  993.73929671]
[ 126.7907155  1840.48894526]
[1458.49446844  241.89872763]
[ 994.589359   1264.53192804]
[1332.88739527   93.21604957]
[ 46.58820829 236.06141758]
[1371.60161993   77.38363693]
[1583.62039074  232.13859559]
[1883.94542483  669.80555718]
[ 451.61697591 1427.08678086]
[332.37811928 318.71099106]
[936.49046672 220.76323293]
[1230.75125839  612.52549496]
[21.78720886 16.31310768]
[492.32031542 364.18832073]
[455.23883927 326.18247482]
[ 728.25269588 1848.30171659]
[118.55168854 116.60884673]
[1042.06590014 1908.10626265]
[564.92714204 296.91312029]
[ 434.34017112 1292.34102269]
[842.26136541 677.9646354 ]
[1539.71852781  943.67945803]
[114.88258511 610.65118673]
[1618.99188718 1670.5945769 ]
[553.65249185 843.20070903]
[1137.37195097  853.36414235]
[360.77448264  69.9742292 ]
[1364.6656647   886.03081934]
[1262.47512467 1927.64732491]
[  47.07

[478.63400921 529.8124312 ]
[849.93410233 982.3936417 ]
[1740.21350208 1347.28479784]
[  62.63945862 1494.70766468]
[ 483.65937079 1912.13497039]
[ 584.57373756 1753.99919642]
[ 878.82380131 1174.6830924 ]
[ 221.29709863 1149.11468774]
[1178.29635601 1844.13243946]
[ 552.97363386 1922.39456209]
[1484.00795774 1011.73584247]
[1001.81816281  330.17367485]
[ 269.0419817  1469.34328063]
[1914.3220547  1924.52156505]
[ 232.83449017 1284.04964255]
[957.47023065 700.46062893]
[ 75.96532416 403.09583717]
[ 399.61780488 1270.32474944]
[1181.99269844  139.78316009]
[297.32846638 856.70777293]
[ 215.55563455 1369.96014681]
[743.91679535 510.90943107]
[1773.82138729 1804.0256891 ]
[1215.43326463 1498.98095769]
[1342.94539338  729.89834425]
[804.15357992 189.60299459]
[1423.69253581   75.8237335 ]
[1539.52166934  402.40720429]
[564.78995277 156.9855543 ]
[1615.57204325 1780.19673339]
[ 757.22877656 1795.86668884]
[95.0612788  85.35246087]
[74.02917704 77.6159311 ]
[ 451.72787285 1050.75757236]
[137

[627.79423871 331.73862045]
[1905.96826761  295.3493517 ]
[805.28708624 587.6349625 ]
[  31.01221939 1742.6555474 ]
[634.13724423 129.53333289]
[969.50737782 541.31051952]
[1241.22731373 1758.64862776]
[ 301.87361721 1512.57020601]
[ 884.86246446 1700.46722044]
[307.12502683 501.34227798]
[1477.04748553  941.42808633]
[876.64348091  29.86036644]
[608.52331228 270.25730828]
[ 676.27056028 1360.24175018]
[499.37828132 994.30718467]
[ 801.39164958 1231.17450282]
[ 961.58995669 1712.19078965]
[519.28707099 320.71850385]
[1505.72607555  861.21122327]
[ 776.46945677 1071.928118  ]
[1499.85845461  760.24102276]
[1519.54649212  379.55051705]
[1808.05728495 1660.4639502 ]
[1139.37743883 1500.51883743]
[332.55381168 333.9532397 ]
[ 528.23497477 1175.13654028]
[1827.54837603  554.99934854]
[ 978.73297974 1932.19582374]
[661.09287527 472.00030329]
[633.75132626 354.88538028]
[1515.58292131 1433.86118979]
[1579.91877105  623.71996147]
[505.52051876 704.43974123]
[1293.94300475  646.94132375]
[1079.

[ 355.0632633  1820.12536807]
[225.15405384 464.79654218]
[1242.98401356 1521.73775539]
[924.36913704 339.46770643]
[213.29109181 932.56905231]
[890.62875762 131.62281312]
[ 861.60721456 1866.16656858]
[ 597.97913432 1672.84967598]
[199.75920544 410.28997579]
[261.6704239  593.21259012]
[1133.48044849 1935.91345743]
[840.07252344  56.8867314 ]
[668.37753805 808.62145915]
[1186.31362745 1321.7087071 ]
[1269.86979642 1484.369155  ]
[1037.13738564  645.15135761]
[ 37.38469987 447.81151437]
[905.16534985 316.73078956]
[1122.05579978  727.71351345]
[ 840.09837471 1094.88972909]
[1086.87241929 1746.69462038]
[540.04160474 108.67599635]
[ 133.69509012 1266.37808994]
[1522.01045283  860.63162118]
[ 788.25032172 1813.77427667]
[1612.63138626  843.69125653]
[631.88365847 599.19028772]
[1213.11251909  949.67503891]
[ 336.56888464 1421.77881756]
[1895.50060124  591.74655253]
[1863.98393513  137.92376821]
[771.03165984  66.26119084]
[ 297.6127434  1608.81602966]
[ 701.58124992 1108.33908268]
[1512.

In [21]:
np.arange(6).reshape((3,2)), np.choose([2,5], np.arange(6).reshape((3,2)))

ValueError: invalid entry in choice array

In [23]:
np.ndarray(5)

array([0.0000000e+000, 7.4057351e-315, 9.8813129e-324,            nan,
       0.0000000e+000])

array([[0.4472136, 0.4472136, 0.4472136, 0.4472136, 0.4472136],
       [0.4472136, 0.4472136, 0.4472136, 0.4472136, 0.4472136],
       [0.4472136, 0.4472136, 0.4472136, 0.4472136, 0.4472136],
       [0.4472136, 0.4472136, 0.4472136, 0.4472136, 0.4472136]])